# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- RN - running
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [38]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER

DAY_OFFS = []
DESIRED_MEAN_VALUE = 275
rep_data, wide_use_data, feedback_data = prepare_data(PATH, DAY_OFFS,
                                              DESIRED_MEAN_VALUE)
month, mean, std, df = wide_use_data
df

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,RN,GM
0,1,work,0,0,43,43,1,2,130,0,0,0
1,2,work,0,265,15,280,1,0,105,0,0,0
2,3,work,0,205,69,274,2,3,165,10,0,0
3,4,work,0,185,115,300,3,5,230,0,13,0
4,5,work,0,0,19,19,4,3,160,10,0,0
5,6,work,0,260,66,326,5,4,210,0,11,0
6,7,work,0,195,114,309,6,4,140,10,0,0
7,8,work,0,50,10,60,7,2,215,0,0,0
8,9,work,0,250,30,280,7,0,120,10,0,0
9,10,work,0,222,22,244,8,4,290,0,15,0


In [39]:
rep_data

([0, 49, 16, 65, 217, 0, 'you studied every day'],
 [['OD', 'LE', 'RN', 'GM'],
  14,
  [54.5, 1.33, 1.08, 0.0],
  [18, 8, 5, 0],
  {1: 0, 2: 2, 3: 5, 4: 5, 5: 2}])

# Working with 'hobby' data

In [40]:
import re
import pandas as pd

PATH = "test.txt"
# PATH = "2023 - What I read, watched and played.txt"
# Read the text file with UTF-8 encoding
with open(PATH, "r", encoding="utf-8") as file:
    content = file.read()

print(content)

Books:
1. "Seneca: Of Tranquility of Mind" by Seneca | audiobook on August 4, 2023
2. "The Hatchet" by Gary Paulsen | audiobook on August 7, 2023


Movies:
1. Romancing the Stone on Jan 6, 2023
2. Star Trek: The Wrath of Khan on Jan 15, 2023
3. Greyhound on Jan 16, 2023
4. The Thing on Jan 19, 2023


Games:
1. Marvel's Guardians of the Galaxy on Jan 15, 2023 (25 hours)
2. Titanfall 2 on February 19, 2023 (6 hours)


In [41]:
# Find the 'Movies:' section using the regular expression
movies_section = re.search(r"Movies:(.*?)(?=\n\n|$)", content, re.DOTALL).group(1).strip()

# Print the extracted movies section
print("Movies Section:")
print(movies_section)

Movies Section:
1. Romancing the Stone on Jan 6, 2023
2. Star Trek: The Wrath of Khan on Jan 15, 2023
3. Greyhound on Jan 16, 2023
4. The Thing on Jan 19, 2023


In [42]:
# Define a regular expression pattern for extracting movie information
movie_pattern = re.compile(r'\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2}), (\d{4})')

# Extract movie information using the pattern
movie_data = movie_pattern.findall(movies_section)

cols = ["Name", "Month", "Day", "Year"]
# Create a pandas DataFrame for movies
movies_df = pd.DataFrame(movie_data, columns=cols)

# Print the DataFrame
print("Movies DataFrame:")
print(movies_df)

Movies DataFrame:
                           Name Month Day  Year
0           Romancing the Stone   Jan   6  2023
1  Star Trek: The Wrath of Khan   Jan  15  2023
2                     Greyhound   Jan  16  2023
3                     The Thing   Jan  19  2023
